# Models

## Import dependencies

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

## Import dataset

In [2]:
df = pd.read_csv("datasets/features_with_politeness.csv", encoding="ISO-8859-1").dropna()

## Explore dataset

In [3]:
print(df.shape)
df.head()

(10162, 48)


,question_id,answer_id,a_score_rel_q_score,answer_score,entities_matches,reputation,reply_by_author,len_answer_text,is_accepted,code_snippet_count,...,Subjectivity,Bare.Command,WH.Questions,YesNo.Questions,Gratitude,Apology,Truth.Intensifier,Affirmation,Adverb.Just,Conjunction.Start
0,22528,22594,-6,5,1,17497.0,False,628,False,0,...,0,0,0,0,0,0,0,0,0,0
1,22528,22587,2,13,0,17787.0,False,699,True,0,...,0,0,0,0,0,0,1,0,0,1
2,22528,22578,-11,0,1,21216.0,False,338,False,1,...,0,2,0,0,0,0,0,0,1,0
3,22528,22551,-10,1,2,480242.0,False,422,False,0,...,0,0,0,0,0,0,0,0,0,0
4,22528,22537,-11,0,0,10583.0,False,318,False,0,...,0,1,1,1,0,0,0,0,0,0


## Split train-test

In [4]:
# What is answer_score?

features = ['a_score_rel_q_score', 'answer_score',
       'entities_matches', 'reputation', 'reply_by_author', 'len_answer_text', 'code_snippet_count', 'link_count', 'Hedges',
       'Positive.Emotion', 'Negative.Emotion',
       'Swearing', 'Negation', 'Informal.Title', 'Reasoning', 'Reassurance', 'Please', 'Agreement', 'Acknowledgement',
       'Subjectivity', 'Bare.Command', 'WH.Questions', 'YesNo.Questions',
       'Gratitude', 'Apology', 'Truth.Intensifier', 'Affirmation']

In [5]:
X = df[features]
y = df['is_accepted']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Train SVC

In [ ]:
svclassifier = SVC(kernel='linear', verbose=True)
svclassifier.fit(X_train, y_train)

[LibSVM]

## Predict `is_accepted`

In [ ]:
y_pred = svclassifier.predict(X_test)
print(y_pred)

## Evaluate model

In [ ]:
print(classification_report(y_test, y_pred))